# Tasks

Tasks are the basis of this library. A task is just a pair of a DatasetLoader and TrainableModel. Currently,
we only support "ClassificationTask"s. 
You create one using a TaskBuilder, ensuring to add a model and dataset loader:

In [ ]:
!pip install rocelib

In [ ]:
from rocelib.datasets.ExampleDatasets import get_example_dataset
from rocelib.tasks.TaskBuilder import TaskBuilder
# Get your dataset
dl = get_example_dataset("ionosphere")

# Using a simple neural net from RoCELib
You can use a simple neural network which is created and trained within our library. We support Pytorch, Keras and Sklearn models:

In [ ]:
#Pytorch Model
ct = TaskBuilder().add_pytorch_model(34, [8], 1, dl).add_data(dl).build()

#Keras Model
# ct = TaskBuilder().add_keras_model(34, [8], 1, dl).add_data(dl).build()

#Sklearn Model
#For sklearn instead of passing in the model dimension, you pass the model type:
#"decision tree", "logistic regression" or "svm":
ct = TaskBuilder().add_sklearn_model("decision tree", dl).add_data(dl).build()

The default task type is a classification task but you can adjust this using the TaskBuilder's ```set_task_type``` function

# Importing your own Model
You can import your own pretrained Pytorch, Keras or Sklearn model to use in our library for recourse generation/evaluation

# Importing a Pytorch Model

To import a .pt or .pth file:
You must save your entire model, rather than just the state dictionary i.e. do:
torch.save(model, "./model.pt”)
Rather than: torch.save(model.state_dict(), "./model.pt”)
To then load it into our library you can create a task using the ```add_model_from_path``` function:

In [ ]:
ct = TaskBuilder().add_model_from_path("model.pt").add_data(dl).build()

# Importing a Keras Model

Use the ```add_model_from_path()``` function, providing a .keras file

In [ ]:
ct = TaskBuilder().add_model_from_path("model.keras").add_data(dl).build()

# Importing a Sklearn Model

Use the ```add_model_from_path()``` function, providing a .pkl file

In [ ]:
ct = TaskBuilder().add_model_from_path("model.pkl").add_data(dl).build()

# Adding Mutliple Models to the Task (for Model Multiplicity Evaluation)
You can add multiple models to the task
Note that the main model will be the first model that you added to the task (i.e. this is the model that counterfactuals will be generated for when you call ```ct.generate```)

In [ ]:
ct = (TaskBuilder()
      .add_pytorch_model(34, [8], 1, dl) #base model
      .add_keras_model(34, [8], 1, dl)
      .add_keras_model(34, [8], 1, dl).add_data(dl).build())


# Naming the imported Models
You can optionally name your models when you import them. If you do not provide a name the library will generate a default name for the model e.g. "pytorch_model_1", "sklearn_model_1", "keras_model_1", "pytorch_model_2" etc.

In [ ]:
ct = (TaskBuilder()
      .add_pytorch_model(34, [8], 1, dl, "my_model")
      .add_keras_model(34, [8], 1, dl)
      .add_keras_model(34, [8], 1, dl).add_data(dl).build())

print(ct.mm_models.keys())

# Generating Counterfactuals for the Base Model
To generate counterfactuals, you use the `ct.generate` method, 
You can pass in the list of recourse methods you would like to generate counterfactuals for as well as any kwargs. (use `ct.get_recourse_methods()` to see all supported recourse methods)
If you do not provide a list of recourse methods, counterfactuals will be generated for all recourse methods
This will store the counterfactuals within the task for use when evaluating later and will also return them as a dictionary from recourse method to a tuple of (Pandas dataframe holding the counterfactual, time taken to generate the counterfactual)

In [ ]:
ct.get_recourse_methods()

In [ ]:
#Generate counterfactuals for MCE and BinaryLinearSearch methods
ces = ct.generate(["MCE", "NNCE"])
print(ces)

In [ ]:
#Generate counterfactuals for all recourse methods
# ces = ct.generate()

In [ ]:
#An example with kwargs:
res = ct.generate(["BinaryLinearSearch"], column_name="HiringDecision")

# Generate Counterfactuals for All Models
To generate counterfactuals for all models, you must have added multiple models to the task, else an error will be printed when you attempt to generate counterfactuals. Then you can use the `ct.generate_mm` method.
You can pass in the list of recourse methods you would like to generate counterfactuals for as well as any kwargs. (use `ct.get_recourse_methods()` to see all supported recourse methods)
If you do not provide a list of recourse methods, counterfactuals will be generated for all recourse methods
This will store the counterfactuals within the task for use when evaluating later and will also return them as a nested dictionary from recourse method to model name to a tuple of (Pandas dataframe holding the counterfactual, time taken to generate the counterfactual)

In [ ]:
ct = TaskBuilder().add_pytorch_model(34, [8], 1, dl).add_keras_model(34, [8], 1, dl).add_data(dl).build() #Ensure your task has multiple models
ces = ct.generate_mm(["MCE", "NNCE"])
print(ces)

# Generating Evaluation Benchmarking Metrics
To generate evaluation metrics, you can use the `ct.evaluate(methods, evaluations)` method, which will print the evaluation metrics in a table.
You can pass in the list of recourse methods you would like to evaluate and evaluation metrics you would like to use

It is important to note that you can only evaluate recourse methods that you have already called `ct.generate` for. 
If you have not already generated counterfactuals for recourse method X, and then try to evaluate X, the library will skip recourse method X and print a message telling you to generate counterfactuals for it first. See the below example

In [ ]:
ct = TaskBuilder().add_pytorch_model(34, [8], 1, dl).add_keras_model(34, [8], 1, dl).add_data(dl).build() #Ensure your task has multiple models
ces = ct.generate(["NNCE"])
#We have not generated counterfactuals for MCE, so when we try to evaluate MCE the library will skip it and only evaluate NNCE
evals = ct.evaluate(methods=["MCE", "NNCE"], evaluations=["Distance"])

In [ ]:
ces = ct.generate(["MCE"])
# Sicne we have generated for MCE, we can now evaluate for MCE
evals = ct.evaluate(methods=["MCE", "BinaryLinearSearch"], evaluations=["Distance"])

If you do not provide the list of recourse methods, the library will evaluate all recourse methods that you have previously called `generate` on.
If you do not provide the list of evaluation metrics, the library will use all applicable evaluation metrics
To see all supported evaluation metrics, use `ct.get_evaluation_metrics` 

In [ ]:
# ces = ct.generate()
# evals = ct.evaluate()

## Model Multiplicity Evaluation Metrics 
If you would like to evaluate model multiplicity robustness, you must have already called `ct.generate_mm` for all the recourse methods you wish to evaluate. Note: `ct.generate` will not suffice as that does not generate for all the stored models, you must call `ct.generate_mm`.  
If you have not already called `generate_mm` for recourse method X, and then try to evaluate MM for X, the library will skip recourse method X and print a message telling you to call `generate_mm` for it first. See the below example

In [ ]:
ct = TaskBuilder().add_pytorch_model(34, [8], 1, dl).add_pytorch_model(34, [8], 1, dl).add_data(dl).build()

ces = ct.generate(["MCE"])
ces = ct.generate_mm(["NNCE"])
# Note we have not called generate_mm for MCE (generate does not count as that does not generate for all stored models)

# We have not generated counterfactuals for MCE, so when we try to evaluate MCE the library will skip it and only evaluate NNCE (as we have called generate_MM for NNCE)
evals = ct.evaluate(methods=["MCE", "NNCE"], evaluations=["Distance"])

# Visualising Your Metrics
To get a more visual representation of your evaluation metrics, run `ct.evaluate` with the `visualisation` flag set to `True`
This will generate a bar chart and, if more than 3 evaluation metrics were used, a radar chart

In [ ]:
ces = ct.generate(["MCE", "NNCE", "KDTreeNNCE", "Wachter", "RNCE"])
evals = ct.evaluate(visualisation=True)

# Adding Your Own Recourse Method
To add your own recourse method, create the class for your recourse method. Your class should extend RecourseGenerator and implement the `_generation_method` method which generates a counterfactual for a single instance.
You should then call `ct.add_recourse_method(method name, method class)`
You can now use the library to generate counterfactuals with your recourse method and evaluate your recourse method

In [ ]:
from rocelib.recourse_methods.RecourseGenerator import RecourseGenerator
import pandas as pd
class MyRecourseMethod(RecourseGenerator):
    def _generation_method(self, instance, column_name="target", neg_value=0, **kwargs):
        ce = pd.DataFrame([[1, 2, 3, 4]], columns=["new_recourse_method_1", "new_recourse_method_2", "new_recourse_method_3", "new_recourse_method_4"])
        return ce
ct.add_recourse_method("my_recourse_method", MyRecourseMethod)
ces = ct.generate(['my_recourse_method'])
print(ces)

# Adding Your Own Evaluation Metric
To add your own evaluation metric, create the class for your evaluation metric. Your class should extend the Evaluator class and implement the `evaluate` method.
You should then call `ct.add_evaluation_metric(metric name, metric class)`
You can now use the library to evaluate recourse methods with this metric

In [ ]:
from rocelib.evaluations.robustness_evaluations.Evaluator import Evaluator
class MyEvaluator(Evaluator):
    def evaluate(self, recourse_method, **kwargs):
        return -1

ct.add_evaluation_metric("my_evaluator", MyEvaluator)
ces = ct.generate(['MCE'])
evals = ct.evaluate(['MCE'], ['my_evaluator'])

# Adding Your Own Robustness Evaluation Metric
To add your own evaluation metric, create the class for your evaluation metric. Your class should extend the corresponding evaluator superclass based on the robustness notion:
IC should extend InputChangesRobustnessEvaluator
MC should extend ModelChangesRobustnessEvaluator
NE should extend NoisyExecutionRobustnessEvaluator
MM should extend ModelMultiplicityRobustnessEvaluator
 
Your class should then implement the `evaluate_single_instance` method.
You should then call `ct.add_evaluation_metric(metric name, metric class)`
You can now use the library to evaluate recourse methods with this robustness metric

In [ ]:
from rocelib.evaluations.robustness_evaluations.ModelChangesRobustnessEvaluator import ModelChangesRobustnessEvaluator

class NewMCEvaluator(ModelChangesRobustnessEvaluator):
    def evaluate_single_instance(self, instance, counterfactual, **kwargs):
        return True


ct.add_evaluation_metric("my_mc_evaluator", MyEvaluator)
ces = ct.generate(['MCE'])
evals = ct.evaluate(['MCE'], ['my_mc_evaluator'])